In [1]:
import pandas as pd
import json

In [2]:
with open("index_rune", "r") as fp:
    idx = json.load(fp)
with open("impact_categories", "r") as fp:
    impact_category = json.load(fp)


In [6]:
ic_lst = []
for ic in impact_category:
    str_temp = '('
    for icc in ic:
        if icc != ic[-1]:
            str_temp += f'{icc}, '
        else:
            str_temp += f'{icc})'
        # print(icc)
    ic_lst.append(str_temp)
    # print(str_temp)



In [4]:
file_name_sub = r'Results\Ananas sub - CONSQ.xlsx'
df = pd.read_excel(file_name_sub)
# Convert JSON strings back to lists for all columns
df = df.applymap(lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else x)
df = df.set_axis(idx)
df.columns = ic_lst
df

C:\Users\ruw\AppData\Local\Temp\2\ipykernel_4220\3576165090.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else x)


,"(EF v3.1 EN15804, acidification, accumulated exceedance (AE))","(EF v3.1 EN15804, climate change, global warming potential (GWP100))","(EF v3.1 EN15804, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe))","(EF v3.1 EN15804, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels)","(EF v3.1 EN15804, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P))","(EF v3.1 EN15804, eutrophication: marine, fraction of nutrients reaching marine end compartment (N))","(EF v3.1 EN15804, eutrophication: terrestrial, accumulated exceedance (AE))","(EF v3.1 EN15804, human toxicity: carcinogenic, comparative toxic unit for human (CTUh))","(EF v3.1 EN15804, human toxicity: non-carcinogenic, comparative toxic unit for human (CTUh))","(EF v3.1 EN15804, ionising radiation: human health, human exposure efficiency relative to u235)","(EF v3.1 EN15804, land use, soil quality index)","(EF v3.1 EN15804, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves))","(EF v3.1 EN15804, ozone depletion, ozone depletion potential (ODP))","(EF v3.1 EN15804, particulate matter formation, impact on human health)","(EF v3.1 EN15804, photochemical oxidant formation: human health, tropospheric ozone concentration increase)","(EF v3.1 EN15804, water use, user deprivation potential (deprivation-weighted water consumption))"
Steel cylinder - sc 1,"[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),...","[['Compresser opration 50L' (unit, GLO, None),..."
nitrous oxide production - sc 1,"[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N...","[['nitrous oxide production' (kilogram, RER, N..."
Use phase basecase - sc 1,"[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 227....","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 1.33...","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]","[['Use phase basecase' (unit, GLO, None), 0.0]]"
EoL 50L cylinder - FU

In [44]:
for index, row in df.iterrows():
    for col in df.columns:
        temp = 0
        for i, proc in enumerate(row[col]):
            if 'transport' in proc[0]:
                # print(index)
                # print(col)
                # print(i, proc[0], proc[1])
                # print(df.loc[proc])
                temp += proc[1]
        print(f'{index} : Transport for {col} = {temp}, {proc}')
            

Steel cylinder - sc 1 : Transport for (EF v3.1 EN15804, acidification, accumulated exceedance (AE)) = 3.804577986184128e-07, ["'nitrous oxide production' (kilogram, RER, None)", 0.0]
Steel cylinder - sc 1 : Transport for (EF v3.1 EN15804, climate change, global warming potential (GWP100)) = 0.00017534877155276442, ["'nitrous oxide production' (kilogram, RER, None)", 0.0]
Steel cylinder - sc 1 : Transport for (EF v3.1 EN15804, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe)) = 0.001171446722622884, ["'nitrous oxide production' (kilogram, RER, None)", 0.0]
Steel cylinder - sc 1 : Transport for (EF v3.1 EN15804, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels) = 0.0024753937442653284, ["'nitrous oxide production' (kilogram, RER, None)", 0.0]
Steel cylinder - sc 1 : Transport for (EF v3.1 EN15804, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P)) = 1.805724951656203e-08, ["'nitrous oxide product